In [4]:
import pandas as pd
from google.cloud import bigquery

from google.oauth2 import service_account


In [5]:
credentials = service_account.Credentials.from_service_account_file("connection-123-892e002c2def.json")

In [6]:


def calculate_macd(credentials):
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)

    query = """
      SELECT
      timestamp,
      ema_9,
      ema_12,
      ema_26
    FROM connection-123.signals.btc_ema
    ORDER BY timestamp
    """
    query_job = client.query(query)
    results = query_job.result()
    df = results.to_dataframe()

    df['macd_line'] = df['ema_12'] - df['ema_26']
    # Calculate signal line (9-period EMA of MACD line)
    df['signal_line'] = df['macd_line'].ewm(span=9, adjust=False).mean()
    # Calculate histogram
    df['histogram'] = df['macd_line'] - df['signal_line']
    # Return only the MACD components
    macd_result = df[['timestamp', 'macd_line', 'signal_line', 'histogram']].copy()

    return macd_result


In [7]:
data = calculate_macd(credentials)

In [8]:
print(data)

      timestamp   macd_line  signal_line   histogram
0    2024-08-11    0.000000     0.000000    0.000000
1    2024-08-12 -166.203063   -33.240613 -132.962451
2    2024-08-13 -250.982308   -76.788952 -174.193356
3    2024-08-14 -214.737850  -104.378732 -110.359119
4    2024-08-15 -332.431995  -149.989384 -182.442611
..          ...         ...          ...         ...
359  2025-08-05  345.445423  1220.450378 -875.004955
360  2025-08-06  186.824156  1013.725134 -826.900978
361  2025-08-07  131.714816   837.323070 -705.608254
362  2025-08-08  281.790819   726.216620 -444.425801
363  2025-08-09  334.328474   647.838991 -313.510516

[364 rows x 4 columns]
